In [2]:
import torch
import torchdata

In [3]:
#install transformrs package by huggingFace
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00


In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00


In [5]:
import transformers

In [6]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [7]:
#Load DialogSum Hugging face Dataset
hugginface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(hugginface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
example_indices = [40,200]

dash_line = '-'.join('' for x in range(100))

for i,index in enumerate(example_indices):
  print(dash_line)
  print('Example ', i+1)
  print('INPUT DIALOGUE')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print('BASELINE HUMAN SUMMARY:')
  print(dataset['test'][index]['summary'])
  print(dash_line)


---------------------------------------------------------------------------------------------------
Example  1
INPUT DIALOGUE
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
Example  2
INPUT DIALOGUE
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm n

In [9]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-base")

# Load FAWN-T5 Model

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [11]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", use_fast=True)

In [12]:
sentence = "what time is it, Tom?"

#encode the sentance into tokens
sentence_encoded = tokenizer(sentence, return_tensors='pt')
print('Sentence Encoded :', sentence_encoded['input_ids'][0])

#decode the sentance

sentence_decoded = tokenizer.decode(
    sentence_encoded['input_ids'][0],
    skip_sepacial_tokens = True
)

print('Sentance Decoded :', sentence_decoded)

Sentence Encoded : tensor([ 125,   97,   19,   34,    6, 3059,   58,    1])
Sentance Decoded : what time is it, Tom?</s>


In [13]:
for i, index in enumerate(example_indices):

  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  input = tokenizer(dialogue, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(input['input_ids'],max_new_tokens = 50)[0],
      skip_special_tokens = True

  )

  print(dash_line)
  print('example ', i+1)
  print(f'INPUT PROMPT: \n {dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY: \n {summary}')
  print(dash_line)
  print(f'MODEL Generation - WITHOUT PROMP ENGINEERING: \n {output} \n')


---------------------------------------------------------------------------------------------------
example  1
INPUT PROMPT: 
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL Generation - WITHOUT PROMP ENGINEERING: 
 Person1: It's ten to nine. 

---------------------------------------------------------------------------------------------------
example  2
INPUT PROMPT: 

In [14]:
for i, index in enumerate(example_indices):

  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  Summarize the following conversation.
  {dialogue}

  Summary:
  """

  input = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(input['input_ids'],max_new_tokens = 50)[0],
      skip_special_tokens = True

  )

  print(dash_line)
  print('example ', i+1)
  print(f'INPUT PROMPT: \n {prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY: \n {summary}')
  print(dash_line)
  print(f'MODEL Generation - WITHOUT PROMP ENGINEERING: \n {output} \n')


---------------------------------------------------------------------------------------------------
example  1
INPUT PROMPT: 
 
  Summarize the following conversation.
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

  Summary:
  
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL Generation - WITHOUT PROMP ENGINEERING: 
 The train is about to leave. 

-----------------------------------------------------------------

In [15]:
#Let's use slightly Deffrent promp
for i,index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  Dialogue: {dialogue}

  what was going on?
  """

  input = tokenizer(prompt, return_tensors = 'pt')
  output = tokenizer.decode(
               model.generate(input['input_ids'],
                              max_new_tokens = 50)[0],
               skip_spatial_tokens = True

  )

  print(dash_line)
  print('example', i+1)
  print(dash_line)
  print(f'input prompt \n : {prompt}')
  print(dash_line)
  print(f"Baseline Human Summary \n : {summary}")
  print(dash_line)
  print(f"Model Generation zero shot \n : {output}")





---------------------------------------------------------------------------------------------------
example 1
---------------------------------------------------------------------------------------------------
input prompt 
 : 
  Dialogue: #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

  what was going on?
  
---------------------------------------------------------------------------------------------------
Baseline Human Summary 
 : #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation zero shot 
 : <pad> Tom is late for the train.

# Summarize the dialogue with 1 shot or few shot inference

In [16]:
def make_prompt(example_indices_full, example_index_to_summarize):
  prompt = ''

  for index in example_indices_full:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt += f"""
    Dialogue:
    {dialogue}
    What was going on:
    {summary}

    """
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
    Dialogue:
    {dialogue}
    What's going on
    """
  return prompt





In [17]:
example_indices_full = [40]
example_indices_to_summarize = 200
one_shot_prompt = make_prompt(example_indices_full,example_indices_to_summarize)
print(one_shot_prompt)


    Dialogue:
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    What was going on:
    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

    
    Dialogue:
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to beg

In [18]:
#now pass the prompt to perform one shot inference

human_summary = dataset['test'][example_indices_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt , return_tensors = 'pt')

summary = tokenizer.decode(model.generate(
    inputs["input_ids"],
    max_new_tokens = 50
)[0],skip_special_tokens = True )

print(dash_line)
print(f"Model output : {summary}")
print(dash_line)
print(f"Human Summary : {human_summary}")

---------------------------------------------------------------------------------------------------
Model output : #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.
---------------------------------------------------------------------------------------------------
Human Summary : #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.


# FEW SHOT INFERENCE

In [26]:
multiple_example_indices = [20,40,55]
few_shot_prompt = make_prompt(multiple_example_indices, 200)
print(few_shot_prompt)


    Dialogue:
    #Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.
    What was going on:
    #Person1# thinks #Person2# has chicken pox and warns #Person2# about the possible hazards but #Person2# thinks it will be fine.

    
    Dialogue:
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm no

In [27]:
human_summary = dataset['test'][200]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
updated_summry = tokenizer.decode(model.generate(
    inputs['input_ids'],
    max_new_tokens = 50
)[0],skip_special_token = True
                                  )

print(dash_line)
print(f"Model output: {updated_summry}")
print(dash_line)
print(f"Human Summary: {human_summary}")

---------------------------------------------------------------------------------------------------
Model output: <pad> #Person1# has considered upgrading his system. #Person2#: Yes, but I'm not sure what exactly I would need. #Person1#: You could consider adding a painting program to your software. It
---------------------------------------------------------------------------------------------------
Human Summary: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.


# Generative configuration parameters for inference

In [31]:
generation_config = GenerationConfig(max_new_tokens = 100, do_sample = True, temprature = 1)
inputs = tokenizer(few_shot_prompt, return_tensors = 'pt')
summary = tokenizer.decode(model.generate(
    inputs['input_ids'],
    generation_config = generation_config
)[0],skip_special_tokens = True)

print(dash_line)
print(f"Model output:{summary}")
print(dash_line)
print(f"Human Summary: {human_summary}")

---------------------------------------------------------------------------------------------------
Model output:#Person1 agrees to make changes to the system (By adding a painting program) so that it's easier to make up of things for advertising. #Person2 agrees to make improvements when updating the software.
---------------------------------------------------------------------------------------------------
Human Summary: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
